# Sherrington-Kirkpatrick Model
Sherrington Kirkpatrick model has the Hamiltonian or cost function

$$ C_J = -\frac{1}{2} \sum_{i\neq j}J_{ij}s_i s_j $$

In this model, $J_{ij}$ are Gaussian distribtued with zero mean and variance of $1/N$, and $s_i$ are Ising spins with values $s_i\in\{+1,-1\}$. Minimization of the cost function, or finding the ground state energy of $C$ is N-p complete problem. 

In the file `apa.py`, the `class` `MeanFieldSK` implements the classical mean field dynamics of a 3-dimensional spin vector under the alternating application of the Hamiltonians,

$$ H_P = -\sum_i h_i \sigma^z_i - \frac{1}{2}\sum_{i\neq j} J_ij \sigma^z_i \sigma^z_j, $$
and
$$ H_D = -\sum_i \Delta_i \sigma^x_i. $$

The constants $\Delta_i > 0$, which ensures that the ground state of $H_D$ is a product state of qubits in $+1$ eigenstate of $\sigma^x$. The alternating sequence arises from the quantum approximate optimization algorithm (QAOA) [2]. The QAOA is a general-purpose algorithm for combinatorial optimization problems more general than the SK model. However, its application to finding the ground state energy of the SK model is a valuable benchmark. The alternating sequence also arises from the quantum adiabatic algorithm, which aims to take the ground state of the solvable Hamiltonian $H_D$ into the one for the difficult case of $H_P$, via the evolution

$$ H(t) = \left(1-\frac{t}{T}\right)H_D + \frac{t}{T}H_P. $$

Trotter-Suzuki scheme then creates a sequence of alternating applications of $H_D$ and $H_P$.

In the article [1], the authors derive a mean-field approximation to the above dynamics. In the mean-field approximation, the total Hamiltonian is

$$ H(t) = -\gamma(t)\sum_i m_i(t)n^z_i(t) -\beta(t)\sum_i \Delta_i n_i^x(t), $$

where $\mathbf{n}_i(t)$ is now a classical 3-vector with unit norm, and they define an effective magnetization,

$$ m_i(t) = \sum_i h_i + \frac{1}{2}\sum_{i\neq j} J_{ij} n^z_h(t). $$

They propose an algorithm in the paper that minimizes $H(t)$ in the limit of large number of time steps. After minimization, they propose that the Ising spins defined by,

$$ s_i \equiv \text{sgn}(n^z_i), $$

are a very good approximation to the minimizer of $C_J$. It is known that in the typical case, the minimimum value achieved is

$$ \lim_{N\rightarrow\infty} \frac{1}{N}\max_{\mathbf{s}}C_J(\mathbf{s}) = -0.763166\ldots $$

In this notebook, I implement the authors' algorithm. I find the minimium of -0.7388, which is close to the one expected. I used $N=200$, a time step of $~0.5$, and executed the dynamics for 10,000 time steps. 

References:<br>
[1. Mean-Field Approximate Optimization Algorithm](https://arxiv.org/abs/2303.00329) <br>
[2. A Quantum Approximate Optimization Algorithm ](https://arxiv.org/abs/1411.4028)<br>
[3. The Quantum Approximate Optimization Algorithm and the Sherrington-Kirkpatrick Model at Infinite Size](https://quantum-journal.org/papers/q-2022-07-07-759/)

In [4]:
from quantum.qprimitives import aoa
import numpy as np
from matplotlib import pyplot as plt
from scipy import optimize
%matplotlib inline

In [5]:
# Cost function
def cost_sigma(obj):
    s = np.sign(obj.nvec[:,2])
    return -(s @ (obj.J @ s))

In [6]:
tau = 0.5
# Optimize the time step, tau
if 1:
    obj=aoa.MeanFieldSK(N=200,Jsq=1,tau=tau, p=10000)
    sol=optimize.minimize_scalar(lambda x : obj.costfunc((x,obj.p)),bracket=[0.4,0.8])
    print('Optimal tau',sol.x)
    tau = sol.x
#
E=[]
for _ in range(10):
    obj=aoa.MeanFieldSK(N=200,Jsq=1,tau=tau, p=10000)
    obj.costfunc((tau,10000))
    E+=[obj.cost_spin()]
#
E = np.array(E)
print(E.mean())    

Optimal tau 1.4692299282557282
-0.7132703732187466
